In [1]:
import tensorflow as tf

import os
import sys
import numpy as np
from scipy.io import savemat
from sklearn.linear_model import SGDClassifier

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# import utils
# import loader

2025-10-22 15:25:09.818789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761161109.832971    7590 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761161109.837368    7590 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761161109.848388    7590 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761161109.848401    7590 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761161109.848402    7590 computation_placer.cc:177] computation placer alr

In [15]:
# Add the root project directory
try:
    base_dir = os.path.dirname(os.path.abspath(__file__))
    project_root = os.path.abspath(os.path.join(base_dir, '..', '..', '..'))
except NameError:
    # Running in Jupyter Notebook
    code_dir = os.getcwd()
    project_root = os.path.abspath(os.path.join(code_dir, '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print(code_dir)
print(project_root)

/home/thien/Code/NTN/Hest_NTN_UDA/JMMD
/home/thien/Code/NTN


In [3]:
from Domain_Adversarial.helper import loader, plotfig, PAD
from Domain_Adversarial.helper.utils import H5BatchLoader
from Domain_Adversarial.helper.utils_GAN import visualize_H, post_val
from helper.utils_GAN import save_checkpoint_ as save_checkpoint

In [4]:
import h5py
import scipy.io

In [5]:
# Generate a (16, 792, 14, 2) matrix with random values
# random_matrix = np.random.randn(16, 132, 14, 2)
# random_matrix.shape

# GAN_model = utils_GAN.GAN(n_subc=132)
# out_put = GAN_model(random_matrix)
# print(out_put.gen_out.shape)
# print(out_put.disc_out.shape)
# print(out_put.extracted_features.shape)

In [ ]:
SNR = 0
# source_data_file_path_label = os.path.abspath(os.path.join(code_dir, '..', 'generatedChan', 'OpenNTN','H_perfect.mat'))
source_data_file_path = os.path.abspath(os.path.join(code_dir, '..', 'generatedChan', 'MATLAB', 'tdlA_tdlC', 'tdlA', 'matlabNTN.mat'))
target_data_file_path = os.path.abspath(os.path.join(code_dir, '..', 'generatedChan', 'MATLAB', 'tdlA_tdlC', 'tdlC', 'matlabNTN.mat'))

In [7]:
norm_approach = 'minmax' # can be set to 'std'
lower_range = -1 
    # if norm_approach = 'minmax': 
        # =  0 for scaling to  [0 1]
        # = -1 for scaling to [-1 1]
    # if norm_approach = 'std': can be any value, but need to be defined
adv_weight=0.005
est_weight=1
domain_weight=0.5  # for Domain Discriminator, 0 for no Domain Discriminator

# snr_start = -25
# snr_step = 5
# snr_end = 25
# SNR = np.arange(snr_start, snr_end+1, snr_step)

# SNR = np.array([0])

# if len(SNR) >1:
#     SNR_txt = f'{snr_start}:{snr_step}:{snr_end}'
# else:
#     SNR_txt = f'{SNR[0]}'
    
# ============ CNN settings ==============
if norm_approach == 'minmax':
    if lower_range == 0:
        norm_txt = 'Using min-max [0 1]'
    elif lower_range ==-1:
        norm_txt = 'Using min-max [-1 1]'
elif norm_approach == 'no':
    norm_txt = 'No'
    
CNN_activation = 'Tanh'
CNN_DropOut = 0.2
if CNN_DropOut != 0:
    dropOut_txt = f'Add p={CNN_DropOut} DropOut'
    
# ============ Adversarial for Domain Discriminator settings ==============
    
    
# # create readme.txt file
# content = f"""Generated by file 'Domain_Adversarial/UDA_CNN_v3.ipynb'.
# 28 GHz fc,
# Source dataset got from {source_data_file_path},
# Target dataset got from {target_data_file_path},
# Learning rate {learning_rate},
# {norm_txt} scaler for each sample
# Using {CNN_activation} as activation function of CNN
# {dropOut_txt}
# ========= For Domain Discriminator ==========
# Extract features after layer {extract_layer} of CNN
# {text_lambda}
# """

In [ ]:
# Paths to save
idx_save_path = loader.find_incremental_filename(code_dir + '/model/GAN','ver', '_', '')

save_model = 0
model_path = code_dir + '/model/GAN/ver' + str(idx_save_path) + '_'
# figure_path = code_dir + '/model/GAN/ver' + str(idx_save_path) + '_/figure'
model_readme = model_path + '/readme.txt'
# figure_readme = figure_path + '/readme.txt'

# if not os.path.exists(os.path.dirname(model_readme)):
#     os.makedirs(os.path.dirname(model_readme))
# # if not os.path.exists(os.path.dirname(figure_readme)):
# #     os.makedirs(os.path.dirname(figure_readme))

# # Open the file in write mode ('w'). If the file does not exist, it will be created.
# with open(model_readme, 'w') as file:
#     # Write the content to the file
#     file.write(content)

# # with open(figure_readme, 'w') as file:
# #     # Write the content to the file
# #     file.write(content)

# print(f"File '{model_readme}'  created and content written.")


## Split dataset

In [9]:


batch_size=16

# ============ Source data ==============
source_file = h5py.File(source_data_file_path, 'r')
H_true_source = source_file['H_perfect']
N_samp_source = H_true_source.shape[0]
print('N_samp_source = ', N_samp_source)

# ============ Target data ==============
target_file = h5py.File(target_data_file_path, 'r')
H_true_target = target_file['H_perfect']
N_samp_target = H_true_target.shape[0]
print('N_samp_target = ', N_samp_target)

N_samp_source =  2048
N_samp_target =  2048


In [10]:
# Store random state 
rng_state = np.random.get_state()

# --- Set a temporary seed for reproducible split ---
np.random.seed(1234)   # any fixed integer seed
# Random but repeatable split
indices_source = np.arange(N_samp_source)
np.random.shuffle(indices_source)
indices_target = np.arange(N_samp_target)
np.random.shuffle(indices_target)
# Restore previous random state (so other code stays random)
np.random.set_state(rng_state)
#
train_size = int(np.floor(N_samp_source * 0.9) // batch_size * batch_size)
val_size = N_samp_source - train_size

# Repeat the indices to match the maximum number of samples
N_samp = max(N_samp_source, N_samp_target) 
indices_source = np.resize(indices_source, N_samp)
indices_target = np.resize(indices_target, N_samp)

# =======================================================
## Divide the indices into training and validation sets
# indices_train_source = indices_source[:train_size]
# indices_val_source   = indices_source[train_size:train_size + val_size]

# indices_train_target = indices_target[:train_size]
# indices_val_target   = indices_target[train_size:train_size + val_size]

# to test code
indices_train_source = indices_source[:96]
indices_val_source = indices_source[2032:]
indices_train_target = indices_target[:96]
indices_val_target = indices_target[2032:]

print('train_size = ', indices_train_source.shape[0])
print('val_size = ', indices_val_source.shape[0])

train_size =  96
val_size =  16


## Data Loader

In [11]:
class DataLoaders:
    def __init__(self, file, indices_train, indices_val, tag='prac', batch_size=32): 
        # tag = 'prac' or 'li' or 'ls'
        self.true_train = H5BatchLoader(file, dataset_name='H_perfect', batch_size=batch_size, shuffled_indices=indices_train)
        self.true_val = H5BatchLoader(file, dataset_name='H_perfect', batch_size=batch_size, shuffled_indices=indices_val)

        self.input_train = H5BatchLoader(file, f'H_{tag}', batch_size=batch_size, shuffled_indices=indices_train)
        self.input_val = H5BatchLoader(file, f'H_{tag}', batch_size=batch_size, shuffled_indices=indices_val)

# Source domain
class_dict_source = {
    'GAN_practical': DataLoaders(source_file, indices_train_source, indices_val_source, tag='prac', batch_size=batch_size),
    'GAN_linear': DataLoaders(source_file, indices_train_source, indices_val_source, tag='li', batch_size=batch_size),
    'GAN_ls': DataLoaders(source_file, indices_train_source, indices_val_source, tag='ls', batch_size=batch_size)
}

# Target domain
class_dict_target = {
    'GAN_practical': DataLoaders(target_file, indices_train_target, indices_val_target, tag='prac', batch_size=batch_size),
    'GAN_linear': DataLoaders(target_file, indices_train_target, indices_val_target, tag='li', batch_size=batch_size),
    'GAN_ls': DataLoaders(target_file, indices_train_target, indices_val_target, tag='ls', batch_size=batch_size)
}

In [12]:
loss_fn_ce = tf.keras.losses.MeanSquaredError()  # Channel estimation loss (generator loss)
loss_fn_bce = tf.keras.losses.BinaryCrossentropy(from_logits=False) # Binary cross-entropy loss for discriminator

# Load  Classes

In [13]:
from helper.utils_GAN import GAN
from helper.utils_GAN import train_step_wgan_gp_jmmd, val_step_wgan_gp_jmmd

# Running Loop

In [17]:
import time
start = time.perf_counter()

n_epochs= 5
epoch_min = 0
epoch_step = 1
# n_epochs= 3
# epoch_min = 0
# epoch_step = 1

sub_folder_ = ['GAN_practical']  # ['GAN_linear', 'GAN_practical', 'GAN_ls']


for sub_folder in sub_folder_:
    print(f"Processing: {sub_folder}")
    w_dist = []
    pad_pca_lda = []
    pad_pca_logreg = []
    pad_pca_svm = []
    linear_interp = False
    if sub_folder == 'GAN_linear':
        linear_interp =True # flag to clip values that go beyond the estimated pilot (min, max)
    ##
    loader_H_true_train_source = class_dict_source[sub_folder].true_train
    loader_H_input_train_source = class_dict_source[sub_folder].input_train
    loader_H_true_val_source = class_dict_source[sub_folder].true_val
    loader_H_input_val_source = class_dict_source[sub_folder].input_val
    
    loader_H_true_train_target = class_dict_target[sub_folder].true_train
    loader_H_input_train_target = class_dict_target[sub_folder].input_train
    loader_H_true_val_target = class_dict_target[sub_folder].true_val
    loader_H_input_val_target = class_dict_target[sub_folder].input_val
    ##
    
    # Distribution of original input training datasets (or before training)    
    plotfig.plotHist(loader_H_input_train_source, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name='source_beforeTrain', percent=100)
    plotfig.plotHist(loader_H_input_train_target, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name='target_beforeTrain', percent=100)
    
    plotfig.plotHist(loader_H_input_train_source, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name='source_beforeTrain', percent=99)
    plotfig.plotHist(loader_H_input_train_target, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name='target_beforeTrain', percent=99)
    
    plotfig.plotHist(loader_H_input_train_source, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name='source_beforeTrain', percent=95)
    plotfig.plotHist(loader_H_input_train_target, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name='target_beforeTrain', percent=95)

    plotfig.plotHist(loader_H_input_train_source, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name='source_beforeTrain', percent=90)
    plotfig.plotHist(loader_H_input_train_target, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name='target_beforeTrain', percent=90)

    # Calculate Wasserstein-1 distance for original input training datasets (before training)
    print("Calculating Wasserstein-1 distance for original input training datasets (before training)...")
    w_dist_epoc = plotfig.wasserstein_approximate(loader_H_input_train_source, loader_H_input_train_target)
    w_dist.append(w_dist_epoc)
    
    # Calculate     PAD for original input training datasets with SVM
    pad_svm = PAD.original_PAD(loader_H_input_train_source, loader_H_input_train_target)
    print(f"PAD = {pad_svm:.4f}")
    
    # Calculate PCA_PAD for original input training datasets with PCA_SVM, PCA_LDA, PCA_LogReg
    X_features_, y_features_ = PAD.extract_features_with_pca(loader_H_input_train_source, loader_H_input_train_target, pca_components=100)
    pad_pca_svm_epoc = PAD.calc_pad_svm(X_features_, y_features_)
    pad_pca_lda_epoc = PAD.calc_pad_lda(X_features_, y_features_)
    pad_pca_logreg_epoc = PAD.calc_pad_logreg(X_features_, y_features_)
    
    pad_pca_lda.append(pad_pca_lda_epoc)
    pad_pca_logreg.append(pad_pca_logreg_epoc)
    pad_pca_svm.append(pad_pca_svm_epoc)
    ## 
    
    if not os.path.exists(os.path.dirname(model_path + '/' + sub_folder +'/')):
        os.makedirs(os.path.dirname(model_path + '/' + sub_folder + '/'))   # Domain_Adversarial/model/_/ver_/{sub_folder}

    train_loss          = [] # (epoch,1)
    train_est_loss      = [] 
    train_disc_loss     = [] 
    train_domain_loss   = []
    train_est_loss_target = []
    #    
    val_loss, val_gan_disc_loss, val_domain_disc_loss,\
    val_est_loss_source, val_est_loss_target, val_est_loss,\
    source_acc, target_acc, acc,\
    nmse_val_source, nmse_val_target, nmse_val = [[] for _ in range(12)]
    #
    H_to_save = {}          # list to save to .mat file for H
    perform_to_save = {}    # list to save to .mat file for nmse, losses,...

    # ✅ Use local GAN class instead of utils_GAN.GAN
    model = GAN(n_subc=312, gen_l2=None, disc_l2=1e-5)  # l2 regularization for generator and discriminator
    # ✅ Fixed indentation issue
    gen_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5, beta_2=0.9)
    disc_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, beta_1=0.5, beta_2=0.9)  # WGAN-GP uses Adam optimizer with beta_1=0.5
    # ❌ No domain optimizer needed for JMMD
    ####
    optimizer = [gen_optimizer, disc_optimizer]  # ✅ Only 2 optimizers for JMMD
    ####
    
    flag = 1 # flag to plot and save H_true
    epoc_pad = []    # epochs that calculating pad (return_features == True)
    for epoch in range(n_epochs):
        # ===================== Training =====================
        loader_H_true_train_source.reset()
        # loader_H_practical_train_source.reset()
        loader_H_input_train_source.reset()
        loader_H_true_train_target.reset()
        # loader_H_practical_train_target.reset()
        loader_H_input_train_target.reset()
                
        # loader_H = [loader_H_practical_train_source, loader_H_true_train_source, loader_H_practical_train_target, loader_H_true_train_target]
        loader_H = [loader_H_input_train_source, loader_H_true_train_source, loader_H_input_train_target, loader_H_true_train_target]
        
        # ✅ Only 2 loss functions needed for JMMD
        loss_fn = [loss_fn_ce, loss_fn_bce]
    
        ##########################
        if epoch in [int(n_epochs * r) for r in [0, 0.25, 0.5, 0.75]] or epoch == n_epochs-1:
            # return_features == return features to calculate PAD
            return_features = True
            epoc_pad.append(epoch)
        else:
            return_features = False

        ##########################
        # ✅ Already correctly calling train_step_wgan_gp_jmmd with proper parameters
        train_step_output = train_step_wgan_gp_jmmd(model, loader_H, loss_fn, optimizer, lower_range=-1, save_features=True,
                                adv_weight=adv_weight, est_weight=est_weight, jmmd_weight=domain_weight, linear_interp=linear_interp)
        
        train_epoc_loss_est        = train_step_output.avg_epoc_loss_est
        train_epoc_loss_d          = train_step_output.avg_epoc_loss_d
        train_epoc_loss_domain     = train_step_output.avg_epoc_loss_domain  # Now contains JMMD loss
        train_epoc_loss            = train_step_output.avg_epoc_loss
        train_epoc_loss_est_target = train_step_output.avg_epoc_loss_est_target
                # train_epoc_loss        = total train loss = loss_est + lambda_jmmd * jmmd_loss
                # train_epoc_loss_est    = loss in estimation network in source domain (labels available)
                # train_epoc_loss_domain = JMMD loss (statistical distribution matching)
                # train_epoc_loss_est_target - just to monitor - the machine can not calculate because no label available in source domain
                # All are already calculated in average over training dataset (source/target - respectively)
        print("Time", time.perf_counter() - start, "seconds")
        # Calculate PAD for the extracted features
        if return_features and (domain_weight!=0):
            features_source_file = "features_source.h5"
            features_target_file = "features_target.h5"
            print(f"epoch {epoch+1}/{n_epochs}")
            ## Calculate PCA_PAD for extracted features with PCA_SVM, PCA_LDA, PCA_LogReg
            X_features, y_features = PAD.extract_features_with_pca(features_source_file, features_target_file, pca_components=100)
            pad_svm_epoc = PAD.calc_pad_svm(X_features, y_features)
            pad_pca_svm.append(pad_svm_epoc)
            #
            pad_lda_epoc = PAD.calc_pad_lda(X_features, y_features)
            pad_pca_lda.append(pad_lda_epoc)
            #
            pad_logreg_epoc = PAD.calc_pad_logreg(X_features, y_features)
            pad_pca_logreg.append(pad_logreg_epoc)
            
            ## Distribution of extracted features
            plotfig.plotHist(features_source_file, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name=f'source_epoch_{epoch+1}', percent=99)
            plotfig.plotHist(features_target_file, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name=f'target_epoch_{epoch+1}', percent=99)
            #
            plotfig.plotHist(features_source_file, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name=f'source_epoch_{epoch+1}', percent=100)
            plotfig.plotHist(features_target_file, fig_show = False, save_path=f"{model_path}/{sub_folder}/Distribution/", name=f'target_epoch_{epoch+1}', percent=100)
            #
            # Calculate Wasserstein-1 distance for extracted features
            # print("Calculating Wasserstein-1 distance for extracted features ...")
            # w_dist_epoc = plotfig.wasserstein_approximate(features_source_file, features_target_file)
            # w_dist.append(w_dist_epoc)
            

            if os.path.exists(features_source_file):
                os.remove(features_source_file)
            if os.path.exists(features_target_file):
                os.remove(features_target_file)
            print("Time", time.perf_counter() - start, "seconds")
            
        
        # Average loss for the epoch
        train_loss.append(train_epoc_loss)
        print(f"epoch {epoch+1}/{n_epochs} Average Training Loss: {train_epoc_loss:.6f}")
        #
        train_est_loss.append(train_epoc_loss_est)
        print(f"epoch {epoch+1}/{n_epochs} Average Estimation Loss (in Source domain): {train_epoc_loss_est:.6f}")
        #
        train_disc_loss.append(train_epoc_loss_d)
        print(f"epoch {epoch+1}/{n_epochs} Average Disc Loss (in Source domain): {train_epoc_loss_d:.6f}")
        #
        train_domain_loss.append(train_epoc_loss_domain)
        print(f"epoch {epoch+1}/{n_epochs} Average JMMD Loss: {train_epoc_loss_domain:.6f}")  # ✅ Updated print message
        #
        train_est_loss_target.append(train_epoc_loss_est_target)
        print(f"epoch {epoch+1}/{n_epochs} For observation only - Average Estimation Loss in Target domain: {train_epoc_loss_est_target:.6f}")
        
        
        # ===================== Evaluation =====================
        loader_H_true_val_source.reset()
        loader_H_input_val_source.reset()
        loader_H_true_val_target.reset()
        loader_H_input_val_target.reset()
        loader_H_eval = [loader_H_input_val_source, loader_H_true_val_source, loader_H_input_val_target, loader_H_true_val_target]

        # ✅ Only 2 loss functions needed for JMMD validation
        loss_fn = [loss_fn_ce, loss_fn_bce]
        
        # eval_func = utils_UDA_FiLM.val_step
        if (epoch==epoch_min) or (epoch+1>epoch_min and (epoch-epoch_min)%epoch_step==0) or epoch==n_epochs-1:
            # ✅ Already correctly calling val_step_wgan_gp_jmmd
            H_sample, epoc_val_return = val_step_wgan_gp_jmmd(model, loader_H_eval, loss_fn, lower_range, 
                                            adv_weight=adv_weight, est_weight=est_weight, jmmd_weight=domain_weight, linear_interp=linear_interp)
            visualize_H(H_sample, H_to_save, epoch, plotfig.figChan, flag, model_path, sub_folder, domain_weight=domain_weight)
            flag = 0  # after the first epoch, no need to save H_true anymore
            
        else:
            # ✅ Already correctly calling val_step_wgan_gp_jmmd
            _, epoc_val_return = val_step_wgan_gp_jmmd(model, loader_H_eval, loss_fn, lower_range, 
                                            adv_weight=adv_weight, est_weight=est_weight, jmmd_weight=domain_weight, linear_interp=linear_interp)
        
        post_val(epoc_val_return, epoch, n_epochs, val_est_loss, val_est_loss_source, val_loss, val_est_loss_target,
            val_gan_disc_loss, val_domain_disc_loss, nmse_val_source, nmse_val_target, nmse_val, source_acc, target_acc, acc, domain_weight=domain_weight)
        
        
        if (epoch==epoch_min) or (epoch+1>epoch_min and (epoch-epoch_min)%epoch_step==0) or epoch==n_epochs-1:
            metrics = {'figLoss': plotfig.figLoss, 'savemat': savemat,
                'train_loss': train_loss, 'train_est_loss': train_est_loss, 'train_domain_loss': train_domain_loss, 'train_est_loss_target': train_est_loss_target,
                'val_est_loss': val_est_loss, 'val_est_loss_source': val_est_loss_source, 'val_loss': val_loss, 'val_est_loss_target': val_est_loss_target,
                'val_gan_disc_loss': val_gan_disc_loss, 'val_domain_disc_loss': val_domain_disc_loss, 'source_acc': source_acc, 'target_acc': target_acc,
                'acc': acc, 'nmse_val_source': nmse_val_source, 'nmse_val_target': nmse_val_target, 'nmse_val': nmse_val,
                'pad_pca_svm': pad_pca_svm, 'pad_pca_lda': pad_pca_lda, 'pad_pca_logreg': pad_pca_logreg, 'epoc_pad': epoc_pad,
                'pad_svm': pad_svm, 'train_disc_loss': train_disc_loss, 'domain_weight': domain_weight, 'optimizer': optimizer}

            save_checkpoint(model, save_model, model_path, sub_folder, epoch, metrics)
    
    # end of epoch loop
    # =====================            
    # Save performances
    # Save H matrix
    savemat(model_path + '/' + sub_folder + '/H_visualize/H_trix.mat', H_to_save)

# end of trainmode   
    

Processing: GAN_practical
Calculating Wasserstein-1 distance for original input training datasets (before training)...
X shape =  (192, 528)
X1 shape =  (96, 528) y1 shape =  (96,)
(96, 528) (96,)
C: 0.01, Error rate: 0.5208
C: 0.1, Error rate: 0.3021
C: 0.5, Error rate: 0.1458
C: 1.0, Error rate: 0.0833
C: 2.0, Error rate: 0.0104
C: 5.0, Error rate: 0.0104
C: 10.0, Error rate: 0.0000
C: 50.0, Error rate: 0.0000
C: 100.0, Error rate: 0.0000
C: 500.0, Error rate: 0.0000
C: 1000.0, Error rate: 0.0000
Best C: 10.0, Best error rate: 0.0000
PAD = 2.0000
PAD = 2.0000
Fitted PCA on batch: source 96/96, target 96/96
Reduced source shape: (96, 100), target shape: (96, 100)
== C: 0.01, Error rate: 0.5208
== C: 0.1, Error rate: 0.4271
== C: 0.5, Error rate: 0.1875
== C: 1.0, Error rate: 0.1354
== C: 2.0, Error rate: 0.0312
== C: 5.0, Error rate: 0.0104
== C: 10.0, Error rate: 0.0000
== C: 50.0, Error rate: 0.0000
== C: 100.0, Error rate: 0.0000
== C: 500.0, Error rate: 0.0000
== C: 1000.0, Error 

I0000 00:00:1761160577.752812    7103 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9553 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
I0000 00:00:1761160577.753406    7103 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9548 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:67:00.0, compute capability: 7.5
I0000 00:00:1761160577.753892    7103 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9486 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5
I0000 00:00:1761160580.506098    7103 cuda_dnn.cc:529] Loaded cuDNN version 90300


Time 18.256228759000805 seconds
epoch 1/5
Fitting IncrementalPCA on batches from features_source.h5 and features_target.h5
Fitted PCA on batch: source 96/96, target 96/96
Reduced source shape: (96, 100), target shape: (96, 100)
== C: 0.01, Error rate: 0.5208
== C: 0.1, Error rate: 0.0104
== C: 0.5, Error rate: 0.0000
== C: 1.0, Error rate: 0.0000
== C: 2.0, Error rate: 0.0000
== C: 5.0, Error rate: 0.0000
== C: 10.0, Error rate: 0.0000
== C: 50.0, Error rate: 0.0000
== C: 100.0, Error rate: 0.0000
== C: 500.0, Error rate: 0.0000
== C: 1000.0, Error rate: 0.0000
Best C: 0.5, Best error rate: 0.0000
============ PAD (SVM) = 2.0000
LDA Error rate: 0.0208
============ PAD (LDA) = 1.9167
Logistic Regression Error rate: 0.0000
============ PAD (LogReg) = 2.0000
Time 20.525604689999454 seconds
epoch 1/5 Average Training Loss: 2.637384
epoch 1/5 Average Estimation Loss (in Source domain): 1.865665
epoch 1/5 Average Disc Loss (in Source domain): 1702.558960
epoch 1/5 Average JMMD Loss: 1.543667

<Figure size 640x480 with 0 Axes>

In [ ]:
pad_pca_svm

[1.75,
 1.9166666666666665,
 1.9166666666666665,
 1.9166666666666665,
 1.9583333333333335,
 1.9583333333333335]

In [ ]:
pad_pca_lda

[0.75, 1.5, 1.2083333333333335, 1.375, 1.4583333333333335, 1.3333333333333335]

In [ ]:
epoc_pad

[0, 1, 2, 3, 4]

In [ ]:
pad_pca_logreg

[0.08333333333333304, 1.9583333333333335, 1.9583333333333335, 2.0, 2.0, 2.0]

In [ ]:
nmse_source_LI = []
nmse_target_LI = []
nmse_source_practical = []
nmse_target_practical = []
nmse_source_LI_GAN = []
nmse_target_LI_GAN = []
nmse_source_practical_GAN = []
nmse_target_practical_GAN = []

SNR_dB_ = np.arange(-15, 1, 5)

-5 practical
-5 linear
